## **Trabajo Práctico Individual n° 2: Aprendizaje Supervisado**
____
__Universidad Tecnológica Nacional, Buenos Aires__\
__Ingeniería Industrial__\
__Ciencia de Datos - Curso I5521__
____

- **Nombre:** Nicolas
- **Apellido:** Ferreyra
- **Legajo:** 1757556

En este trabajo práctico aplicaremos algunas técnicas aprendidas para el aprendizaje automático supervisado.

Utilizaremos un dataset de airbnb donde buscaremos predecir el precio de las noches de cada hogar en funcion de algunas variables.

#### Variables:
- Unnamed: 0: Variable de contador de registros (no será utilizada en nuetro análisis)
- neighbourhood: Barrio donde se encuentra ubicado la casa.
- room_type: Tipo de hogar en alquiler.
- price: Precio por noche.
- minimum_nights: Cantidad minimas de noches que pueden alquilar el hogar.
- number_of_reviews: Cantidad de reviews
- reviews_per_month: Tasa de reviews que tienen por mes
- calculated_host_listings_count: Cantidad de hogares ofrecidos por el host.
- availability_365: Cantidad de dias que se encuentra en alquiler el hogar

## **Objetivos del trabajo practico**
- resolver los requisitos que se detallan en las proximas celdas
- dejar comentarios # en cada linea de codigo que se agregue
- fecha limite de entrega: 13 de Octubre 2022 a las 23h59

In [1]:
## Importo librerías de maniupulación de datos
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
## Importo librerías de Aprendizaje automático
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR, LinearSVR
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV

##### **Objetivo 1**
Lo primero que deberan hacer será importar
- el dataset bajo en nombre "london"
- corroborar que lo hayamos cargado correctamente.

In [5]:
# Importo el dataset
london = pd.read_csv('./TP2_airbnb_london.csv', delimiter=',',encoding='latin-1')  # Uso PATH relativo

In [7]:
# Verifico que se haya cargado correctamente
london.head()

,Unnamed: 0,neighbourhood,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,0,Haringey,Private room,35,1,131,1.71,4,262
1,1,Ealing,Private room,70,2,2,0.07,6,62
2,2,Islington,Private room,45,1,14,0.14,2,364
3,5,Tower Hamlets,Entire home/apt,102,5,7,0.07,56,295
4,6,Barnet,Private room,29,5,114,1.13,3,285


##### **Objetivo 2**
Ahora imprimir también las dimensiones del dataset y eliminar la variable "Unnamed:0" del dataset.
Volver a guardar el dataset sin esta variable bajo el nombre "london"

In [9]:
# Dimensiones del dataset. Formato Filas x Columnas
print(str(np.shape(london)[0]) + " Filas x " + str(np.shape(london)[1]) +" Columnas")

51827 Filas x 9 Columnas


In [11]:
# Elimino la variable 'Unnamed: 0' 
london.drop('Unnamed: 0',axis=1,inplace=True)

# Verifico haber eliminado correctamente la variable
london.head()

,neighbourhood,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,Haringey,Private room,35,1,131,1.71,4,262
1,Ealing,Private room,70,2,2,0.07,6,62
2,Islington,Private room,45,1,14,0.14,2,364
3,Tower Hamlets,Entire home/apt,102,5,7,0.07,56,295
4,Barnet,Private room,29,5,114,1.13,3,285


##### **Objetivo 3**
Corroborar que el dataset no cuenta con valores nulos, si los hubiera llenarlos todos con 0.

In [13]:
# Verifico si hay valores nulos
# london.info()
# Verifico si hay valores nulos
print("Variable | Valor nulo?")
london.isnull().any()

Variable | Valor nulo?


neighbourhood                     False
room_type                         False
price                             False
minimum_nights                    False
number_of_reviews                 False
reviews_per_month                 False
calculated_host_listings_count    False
availability_365                  False
dtype: bool

Como no hay valores nulos no es necesario llenar nada con 0

##### **Objetivo 4**
Mostrar el tipo de dato para las variables "neighbourhood" y "room_type".

In [15]:
# Muestro el tipo de dato para una variable definida
print("Variable | Tipo de dato")
print(london[['neighbourhood', 'room_type']].dtypes)

Variable | Tipo de dato
neighbourhood    object
room_type        object
dtype: object


##### **Objetivo 5**
Generar las variables dummies para estas dos variables categóricas y unirlas al dataset london.


In [17]:
## Genero las dummies para la variable neighbourhood, dentro de la variable neigh_dummies.
#neigh_dummies = pd.get_dummies(london['neighbourhood'])
# drop_first=True  revisar, en clase no lo usamos

neigh_dummies = pd.get_dummies(london['neighbourhood'])

Imprimir las dimensiones y primeras lineas de estas dummies

In [19]:
# Dimensiones de neigh_dummies
neigh_dummies.shape

(51827, 33)

In [21]:
# Primeras 5 lineas de neigh_dummies
neigh_dummies.head()

,Barking and Dagenham,Barnet,Bexley,Brent,Bromley,Camden,City of London,Croydon,Ealing,Enfield,...,Merton,Newham,Redbridge,Richmond upon Thames,Southwark,Sutton,Tower Hamlets,Waltham Forest,Wandsworth,Westminster
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
4,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [23]:
## Genero las dummies para la variable room_type, dentro de la variable room_dummies.
room_dummies = pd.get_dummies(london['room_type'])

Imprimir las dimensiones y primeras lineas de estas dummies

In [25]:
# Dimensiones de room_dummies
room_dummies.shape

(51827, 3)

In [27]:
# Primeras 5 lineas de room_dummies
room_dummies.head()

,Entire home/apt,Private room,Shared room
0,False,True,False
1,False,True,False
2,False,True,False
3,True,False,False
4,False,True,False


Joinear las variables dummies al dataset



In [29]:
#london = london.join(room_dummies,neigh_dummies)  Esto no los dieron ellos pero da error
# Joinear las variables dummies al dataset london
london = pd.concat([london, neigh_dummies, room_dummies], axis=1)

In [ ]:
#london = pd.concat([london.drop(['neighbourhood', 'room_type'], axis=1), neigh_dummies, room_dummies], axis=1)


In [31]:
london.head()

,neighbourhood,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,Barking and Dagenham,Barnet,...,Richmond upon Thames,Southwark,Sutton,Tower Hamlets,Waltham Forest,Wandsworth,Westminster,Entire home/apt,Private room,Shared room
0,Haringey,Private room,35,1,131,1.71,4,262,False,False,...,False,False,False,False,False,False,False,False,True,False
1,Ealing,Private room,70,2,2,0.07,6,62,False,False,...,False,False,False,False,False,False,False,False,True,False
2,Islington,Private room,45,1,14,0.14,2,364,False,False,...,False,False,False,False,False,False,False,False,True,False
3,Tower Hamlets,Entire home/apt,102,5,7,0.07,56,295,False,False,...,False,False,False,True,False,False,False,True,False,False
4,Barnet,Private room,29,5,114,1.13,3,285,False,True,...,False,False,False,False,False,False,False,False,True,False


In [33]:
# Ver las primeras filas del dataset resultante
print(london.head())

   neighbourhood        room_type  price  minimum_nights  number_of_reviews  \
0       Haringey     Private room     35               1                131   
1         Ealing     Private room     70               2                  2   
2      Islington     Private room     45               1                 14   
3  Tower Hamlets  Entire home/apt    102               5                  7   
4         Barnet     Private room     29               5                114   

   reviews_per_month  calculated_host_listings_count  availability_365  \
0               1.71                               4               262   
1               0.07                               6                62   
2               0.14                               2               364   
3               0.07                              56               295   
4               1.13                               3               285   

   Barking and Dagenham  Barnet  ...  Richmond upon Thames  Southwark  Sutton  \

##### Dividiremos nuestro dataset en la variable dependiente "y" (label o etiqueta) y las variables independientes "x"

In [35]:
y = np.array(london[['price']])
x = london.drop(['price', 'neighbourhood','room_type'], axis=1)

In [37]:
y.shape
print(y)


[[ 35]
 [ 70]
 [ 45]
 ...
 [280]
 [ 35]
 [ 66]]


##### **Objetivo 7**

Dividir el dataset entre Train y Test, tomar un test_size del 30%.

In [69]:
xtrain, xtest, ytrain, ytest = train_test_split(x.values, y, test_size=0.3, random_state=4)

In [71]:
# Ver las formas de los conjuntos divididos
print(f'xtrain shape: {xtrain.shape}')
print(f'xtest shape: {xtest.shape}')
print(f'ytrain shape: {ytrain.shape}')
print(f'ytest shape: {ytest.shape}')

xtrain shape: (36278, 41)
xtest shape: (15549, 41)
ytrain shape: (36278, 1)
ytest shape: (15549, 1)


##### **Objetivo 8**
Con el StandardScaler generado en la linea siguiente, realizar el transform para el xtrain y xtest guardandolos en nuevas variables llamadas "xtrain_scal" y "xtest_scal".


In [73]:
scaler = preprocessing.StandardScaler().fit(xtrain)

In [75]:
# Transformo los datos de entrenamiento con la regla de scaling del paso anterior
xtrain_scal = scaler.transform(xtrain)

In [77]:
# Transformo los datos de test con la regla de scaling del paso anterior
xtest_scal = scaler.transform(xtest)

##### **Objetivo 9**
Definir un modelo de regresion lineal standard y entrenarlo para nuestro dataset (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

Obtener las predicciones y el "mean_squared_error" para el test set.

In [79]:
# Creo una instancia del modelo de regresión lineal.
lr = LinearRegression()

In [81]:
# Entreno el modelo
lr.fit(xtrain_scal, ytrain)

# Guardo las predicciones en ypred con los datos de test
ypred = lr.predict(xtest_scal)

In [83]:
# Calculo el error cuadratico medio
np.sqrt(mean_squared_error(ytest, ypred))

39.325278574647704

##### **Objetivo 10**
Definir un modelo de regresion SVM Regressor (https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html) e implementar GridSearch para realizar la búsqueda de hiperparámetros.

Asignar dos posibles valores para el parámetro "kernel", y tres posibles valores para "C" y "gamma".

Setear al parámetro "refit" en True y hacer 5 CrossValidation.

In [85]:
# Defino el modelo de regresión SVR
svreg = SVR()

In [87]:
# Defino los parámetros para la búsqueda de hiperparámetros
param_svreg = {
    'kernel': ['linear', 'rbf'],  # Posibles valores para kernel
    'C': [0.1, 1, 10],            # Posibles valores para C
    'gamma': [0.1, 0.5, 1]   # Posibles valores para gamma
}


In [89]:
# Defino el GridSearch con 5-fold (cv) cross-validation y refit=True
grid_search = GridSearchCV(svreg, param_grid=param_svreg, cv=5, refit=True)

In [91]:
print(xtrain_scal.shape)
print(ytrain.shape)

(36278, 41)
(36278, 1)


In [93]:
# Entrenar el GridSearch con el conjunto de entrenamiento
grid_search.fit(xtrain_scal, ytrain.ravel())

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [0.1, 1, 10], 'gamma': [0.1, 0.5, 1],
                         'kernel': ['linear', 'rbf']})

##### **Objetivo 11**

Mostrar cuales fueron los mejores hiperparámetros y para estos obtener el mean_squared_error en el test set del punto anterior

In [65]:
# Imprimir los mejores hiperparámetros y el mejor modelo
print("Mejores hiperparámetros:", grid_search.best_params_)
print("Mejor score:", grid_search.best_score_)

Mejores hiperparámetros: {'C': 10, 'gamma': 0.1, 'kernel': 'linear'}
Mejor score: 0.4457282407015012


In [67]:
# 2. Realizar predicciones en el conjunto de prueba
ypred = grid_search.predict(xtest_scal)

# 3. Calcular el mean squared error
mse = mean_squared_error(ytest, ypred)
print("Mean Squared Error en el conjunto de prueba:", mse)

Mean Squared Error en el conjunto de prueba: 1779.1285936734978


##### **Objetivo 12**
Repetir el objetivo 10 pero ahora seteando nuevos valores para los hiperparámetros mencionados con el objetivo de mejorar las predicciones.

Cuales son los mejores hiperparámetros ahora?
Obtenemos una mejora en las predicciones?
Mostrar los resultados.